In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -r /content/drive/MyDrive/ML/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.5/123.5 kB 7.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 112.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 8

In [ ]:
import os
import requests
from transformers import BlipProcessor, BlipForQuestionAnswering
from datasets import load_dataset
import torch
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm
import pickle
import numpy as np
from torch.nn.utils import clip_grad_norm_

In [ ]:
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
torch.cuda.empty_cache()
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
for module in model.modules():
    if isinstance(module, torch.nn.Dropout):
        module.p = 0.3

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
from torchvision import transforms
from PIL import Image
import os
import json
import torch

class LandmarkVQADataset(torch.utils.data.Dataset):
    """Custom VQA dataset for landmark images and shared questions."""
    def __init__(self, data_dir, processor, max_answer_length=20):
        self.data = []
        self.processor = processor
        self.max_answer_length = max_answer_length


        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.3),
            transforms.ColorJitter(brightness=0.2, contrast=0.2)
        ])

        for landmark_dir in os.listdir(data_dir):
            landmark_path = os.path.join(data_dir, landmark_dir)
            if not os.path.isdir(landmark_path):
                continue

            with open(os.path.join(landmark_path, 'data.json'), 'r') as f:
                questions_data = json.load(f)

            for image_file in os.listdir(landmark_path):
                if image_file.endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(landmark_path, image_file)
                    for qa in questions_data['questions']:
                        self.data.append({
                            "image_path": image_path,
                            "question": qa['question'],
                            "answer": qa['answer']
                        })

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        data_point = self.data[idx]
        try:
            image = Image.open(data_point["image_path"]).convert("RGB")
            image = self.transform(image)
        except Exception as e:
            print(f"Error loading image {data_point['image_path']}: {e}")
            return None

        question = data_point["question"]
        answer = data_point["answer"]

        encoding = self.processor(image, question, padding="max_length", truncation=True, return_tensors="pt")
        labels = self.processor.tokenizer.encode(answer, max_length=self.max_answer_length, padding="max_length", truncation=True, return_tensors="pt")

        encoding["labels"] = labels.squeeze()

        for k, v in encoding.items():
            encoding[k] = v.squeeze()

        return encoding


In [ ]:
!unzip /teamspace/studios/this_studio/train_data_small.zip

unzip:  cannot find or open /teamspace/studios/this_studio/train_data_small.zip, /teamspace/studios/this_studio/train_data_small.zip.zip or /teamspace/studios/this_studio/train_data_small.zip.ZIP.


In [ ]:
import json
from torchvision.transforms import RandomHorizontalFlip, ColorJitter
data_dir = '/teamspace/studios/this_studio/train_data_fr'
full_dataset = LandmarkVQADataset(data_dir=data_dir, processor=processor)

# Stratified split
train_ratio = 0.9
indices = torch.randperm(len(full_dataset))
train_size = int(train_ratio * len(full_dataset))
train_indices = indices[:train_size]
valid_indices = indices[train_size:]

train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
valid_dataset = torch.utils.data.Subset(full_dataset, valid_indices)

# Reduced batch size for better stability
batch_size = 2
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=4)

print("Training set size: {}, Validation set size: {}".format(len(train_dataset), len(valid_dataset)))


Training set size: 19954, Validation set size: 2218


In [ ]:
patience = 15  # Increased patience
min_eval_loss = float("inf")
early_stopping_hook = 0
best_f1 = 0.0
tracking_information = []
scaler = torch.cuda.amp.GradScaler()
num_epochs = 50
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
# Warm-up scheduler combined with cosine annealing
from transformers import get_linear_schedule_with_warmup
num_training_steps = len(train_dataloader) * num_epochs
num_warmup_steps = num_training_steps // 10
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps
)



# EMA model for stability
from copy import deepcopy
ema_model = deepcopy(model)
ema_decay = 0.999

def update_ema_model(model, ema_model, decay):
    with torch.no_grad():
        for ema_param, param in zip(ema_model.parameters(), model.parameters()):
            ema_param.data.mul_(decay).add_(param.data, alpha=1 - decay)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score
checkpoint_dir="/teamspace/studios/this_studio/checkpoints"
for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()

    # Training loop
    for step, batch in enumerate(tqdm(train_dataloader, desc=f'Training Epoch {epoch+1}')):
        input_ids = batch.pop('input_ids').to(device)
        pixel_values = batch.pop('pixel_values').to(device)
        attention_mask = batch.pop('attention_mask').to(device)
        labels = batch.pop('labels').to(device)

        # Clear gradients
        optimizer.zero_grad()

        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        scaler.scale(loss).backward()

        # Gradient clipping
        scaler.unscale_(optimizer)
        clip_grad_norm_(model.parameters(), max_norm=1.0)

        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        # Update EMA model
        update_ema_model(model, ema_model, ema_decay)

        epoch_loss += loss.item()

    avg_train_loss = epoch_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} Training Loss: {avg_train_loss:.4f}")

    # Validation loop with EMA model
    ema_model.eval()
    eval_loss, all_preds, all_labels = 0, [], []

    with torch.no_grad():
        for batch in tqdm(valid_dataloader, desc=f'Validating Epoch {epoch+1}'):
            input_ids = batch.pop('input_ids').to(device)
            pixel_values = batch.pop('pixel_values').to(device)
            attention_mask = batch.pop('attention_mask').to(device)
            labels = batch.pop('labels').to(device)

            with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
                outputs = ema_model(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, labels=labels)

            eval_loss += outputs.loss.item()

            generated_ids = ema_model.generate(input_ids=input_ids, pixel_values=pixel_values, attention_mask=attention_mask, max_length=20)
            predictions = processor.batch_decode(generated_ids, skip_special_tokens=True)
            true_labels = processor.batch_decode(labels, skip_special_tokens=True)

            all_preds.extend(predictions)
            all_labels.extend(true_labels)

    avg_eval_loss = eval_loss / len(valid_dataloader)
    exact_match_acc = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, average='weighted')

    print(f"Epoch {epoch+1} - Validation Loss: {avg_eval_loss:.4f} - Accuracy: {exact_match_acc:.4f} - F1 Score: {f1:.4f}")

    # Save best model based on F1 score
    if f1 > best_f1:
        best_f1 = f1
        best_checkpoint_dir = os.path.join(checkpoint_dir, "best_model")
        os.makedirs(best_checkpoint_dir, exist_ok=True)
        ema_model.save_pretrained(best_checkpoint_dir)
        early_stopping_hook = 0
    else:
        early_stopping_hook += 1
        if early_stopping_hook > patience:
            print("Early stopping triggered.")
            break

    # Save periodic checkpoints
    if (epoch + 1) % 5 == 0:
        epoch_checkpoint_dir = os.path.join(checkpoint_dir, f"checkpoint_epoch_{epoch+1}")
        os.makedirs(epoch_checkpoint_dir, exist_ok=True)
        ema_model.save_pretrained(epoch_checkpoint_dir)
        torch.save(optimizer.state_dict(), os.path.join(epoch_checkpoint_dir, 'optimizer.pt'))
        torch.save(scheduler.state_dict(), os.path.join(epoch_checkpoint_dir, 'scheduler.pt'))

    torch.cuda.empty_cache()

Training Epoch 1:   0%|          | 0/9977 [00:00<?, ?it/s]/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Training Epoch 1: 100%|██████████| 9977/9977 [1:21:18<00:00,  2.05it/s]


Epoch 1 Training Loss: 4.2120


Validating Epoch 1: 100%|██████████| 1109/1109 [08:31<00:00,  2.17it/s]


Epoch 1 - Validation Loss: 1.9669 - Accuracy: 0.0712 - F1 Score: 0.0521


Training Epoch 2: 100%|██████████| 9977/9977 [1:21:20<00:00,  2.04it/s]


...

Training Epoch 20: 100%|██████████| 9977/9977 [1:21:23<00:00,  2.13it/s]


Epoch 20 Training Loss: 1.3410


Validating Epoch 20: 100%|██████████| 1109/1109 [08:03<00:00,  2.29it/s]


Epoch 20 - Validation Loss: 1.3705 - Accuracy: 0.9396 - F1 Score: 0.9380


Early stopping triggered.

In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering
from transformers import BlipProcessor, BlipForQuestionAnswering
import requests
from PIL import Image
import json, os, csv
import logging
from tqdm import tqdm
import torch

In [ ]:
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
model = BlipForQuestionAnswering.from_pretrained("/content/drive/MyDrive/ML/Model").to("cuda")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# Create a list to store results
results = []
correct_count = 0  # To count correct predictions
total_count = 0  # Total number of samples

test_data_dir = "/content/drive/MyDrive/ML/test_data"
# Iterate through each test sample directory
samples = os.listdir(test_data_dir)
for filename in tqdm(samples, desc="Processing"):
    sample_path = os.path.join(test_data_dir, filename)

    # Read JSON data
    json_path = os.path.join(sample_path, "data.json")
    with open(json_path, "r") as json_file:
        data = json.load(json_file)
        question = data["question"]
        correct_answer = data.get("answer", "").lower()  # Load the correct answer for accuracy
        image_id = data["id"]

    # Read the corresponding image
    image_path = os.path.join(sample_path, f"{image_id}.jpg")
    image = Image.open(image_path).convert("RGB")

    # Prepare inputs
    encoding = processor(image, question, return_tensors="pt").to("cuda")

    # Generate answer
    with torch.no_grad():
        num_beams = 3 if len(question.split()) < 10 else 5
        out = model.generate(**encoding, max_length=40, num_beams=num_beams)
        generated_text = processor.decode(out[0], skip_special_tokens=True).lower()

    # Calculate accuracy
    if correct_answer:  # Ensure there is a ground-truth answer
        total_count += 1
        if generated_text.strip() == correct_answer.strip():
            correct_count += 1

    # Append result
    results.append((image_id, generated_text, correct_answer))

# Save results to CSV
csv_file_path = "Results/results.csv"
os.makedirs(os.path.dirname(csv_file_path), exist_ok=True)
with open(csv_file_path, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(["ID", "Generated Answer", "Correct Answer"])
    csv_writer.writerows(results)

# Print accuracy
if total_count > 0:
    accuracy = (correct_count / total_count) * 100
    print(f"Accuracy: {accuracy:.2f}%")
else:
    print("No ground-truth answers provided for accuracy calculation.")

print(f"Results saved to {csv_file_path}")


Processing: 100%|██████████| 132/132 [02:42<00:00,  1.32s/it]

Accuracy: 87.10%
Results saved to Results/results.csv
